In [1]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload

# Tests of new features

Use this notebook to test some modifications on the classes without the need to reinstall the package 

In [13]:
import sys
sys.path.insert(0,'/home/marco/Applications/MPPI/mppi/InputFiles/')
sys.path.insert(0,'/home/marco/Applications/MPPI/mppi/Calculators/')
sys.path.insert(0,'/home/marco/Applications/MPPI/mppi/Datasets/')
import InputFiles as I, Calculators as C, Datasets as D

In [14]:
reload(I)
input = I.PwIn()

In [16]:
k = 6
ecut = 80
a0 = 1.42
planeSep  = 10.0
A = [0.,0.,0.]
B = [0.,a0,0.]
a1 = [a0*np.sqrt(3),0.,0.]
a2 = [a0*np.sqrt(3)/2,a0*3./2,0.]
a3 = [0.,0.,planeSep]

input.set_pseudo_dir('../pseudos')
input.system['ibrav'] = 0
input.cell_parameters =[a1,a2,a3]
input.set_atoms_type(1)
input.set_atoms_number(2)
input.set_atoms_position(pos_type='angstrom',pos_list=[['C',A],['C',B]])
input.atypes = {'C': [12.011,'C_pbe-20082014.UPF']}
input.set_convergence_thr(1e-8)
prefix = 'k'+str(k)+'_ecut'+str(ecut)
input.set_prefix(prefix)
input.set_kpoints(ktype='automatic',kpoints=[k,k,1])
input.set_energy_cutoff(ecut)

print(input)

&control
          wf_collect = .true.
        restart_mode = 'from_scratch'
           verbosity = 'high'
          pseudo_dir = '../pseudos'
              prefix = 'k6_ecut80'
/&end
&system
               ibrav = 0
                ntyp = 1
                 nat = 2
             ecutwfc = 80
/&end
&electrons
            conv_thr = 1e-08
/&end
ATOMIC_SPECIES
   C   12.011   C_pbe-20082014.UPF
ATOMIC_POSITIONS { angstrom }
  C   0.0000000000   0.0000000000   0.0000000000
  C   0.0000000000   1.4200000000   0.0000000000
K_POINTS { automatic }
  6  6  1  0  0  0
CELL_PARAMETERS angstrom
  2.4595121467   0.0000000000   0.0000000000 
  1.2297560734   2.1300000000   0.0000000000 
  0.0000000000   0.0000000000  10.0000000000 



In [17]:
import os
test_dir = 'test_dir'
if not os.path.isdir(test_dir): os.mkdir(test_dir)

In [18]:
input.write(test_dir+'/'+'input.in')

The instance of PwIn can be created also from an existing file

In [19]:
reload(I)
input2 = I.PwIn(test_dir+'/'+'input.in')

In [20]:
print(input2)

&control
          wf_collect = .true.
        restart_mode = 'from_scratch'
           verbosity = 'high'
          pseudo_dir = '../pseudos'
              prefix = 'k6_ecut80'
/&end
&system
               ibrav = 0
                ntyp = 1
                 nat = 2
             ecutwfc = 80
/&end
&electrons
            conv_thr = 1e-08
/&end
ATOMIC_SPECIES
   C   12.011   C_pbe-20082014.UPF
ATOMIC_POSITIONS { angstrom }
  C   0.0000000000   0.0000000000   0.0000000000
  C   0.0000000000   1.4200000000   0.0000000000
K_POINTS { automatic }
  6  6  1  0  0  0
CELL_PARAMETERS angstrom
  2.4595121467   0.0000000000   0.0000000000 
  1.2297560734   2.1300000000   0.0000000000 
  0.0000000000   0.0000000000  10.0000000000 

